# Nova seção

In [ ]:
import numpy as np
import os
from PIL import Image
import pickle 
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from google.colab import drive

drive.mount('/content/drive')
np.seterr(divide='ignore', invalid='ignore')

In [5]:
def normalize(array):
    array_min, array_max = array.min(), array.max()
    return ((array - array_min)/(array_max - array_min))

def __img(caminho):
    img = Image.open(caminho)
    img1 = np.array(img)
    img1 = normalize(img1)
    return img1

def ndvi(banda4, banda5, limit1, limit2):
    b4, b5 = __img(banda4)[1500:6500,1500:6500], __img(banda5)[1500:6500,1500:6500]
    resposta = (b5-b4)/(b5+b4)
    resposta[(resposta < limit1)&(resposta > limit2)] = 1000
    return resposta

def hot(banda2, banda4, limit):
    b2, b4 = __img(banda2)[1500:6500,1500:6500], __img(banda4)[1500:6500,1500:6500]
    resposta = b2 - (b4/2) - 0.15
    resposta[(resposta > limit)] = 1000
    return resposta

def create_rgb(r,g,b):
    rgb = np.dstack((__img(r)[1500:6500,1500:6500], __img(g)[1500:6500,1500:6500], __img(b)[1500:6500,1500:6500]))
    return rgb

metods = ['ndvi', 'hot']

tif = ['/tif_4/LC08_L1TP_220079_20190621_20190704_01_T1_B%s.tif',
        '/tif_2/LC08_L1TP_220081_20200404_20200410_01_T1_B%s.tif']

tiflimit = [[-0.1095,-0.154],[-0.00745,-0.0254]]
tiflimithot = [0.098]

path = '/content/drive/MyDrive'

for met in metods:
    cont = 1
    for item in tif:

        bandas = [path + item % x for x in ['2','3','4','5']]
        band2, band3, band4, band5 = bandas[0], bandas[1], bandas[2], bandas[3]
        rgb = create_rgb(band4, band3, band2)

        if met == 'ndvi':
          if band2 == '/content/drive/MyDrive/tif_4/LC08_L1TP_220079_20190621_20190704_01_T1_B2.tif':
            fim = ndvi(band4, band5, tiflimit[0][0],tiflimit[0][1])
          elif band2 == '/content/drive/MyDrive/tif_2/LC08_L1TP_220081_20200404_20200410_01_T1_B2.tif':
            fim = ndvi(band4, band5, tiflimit[1][0],tiflimit[1][1])
        else:
          fim = hot(band2, band4, tiflimithot[0])

        fim[fim == 1000] = 1
        fim[fim != 1] = 0
        rgbX = np.reshape(rgb, (1,(rgb.shape[0]* rgb.shape[1]),3))[0]
        rgbY = np.reshape(fim, (1,(rgb.shape[0]* rgb.shape[1]),1))[0]
        
        if cont == 1:
            rgbX1 = np.copy(rgbX)
            rgbY1 = np.copy(rgbY)
            cont = cont + 1
        else:
            rgbX1 = np.ma.concatenate([rgbX1, rgbX])
            rgbY1 = np.ma.concatenate([rgbY1, rgbY])
            cont = cont + 1

    x, x_t, y, y_t = train_test_split(rgbX1, rgbY1, test_size = 0.3, random_state=101)
    y = y.ravel()

    if met == 'ndvi': 
        print('------NDVI MODEL------')
        logit_model = LogisticRegression(C=0.1, verbose=0, max_iter=1000)
        logit_model.fit(x, y)
        y_pred = logit_model.predict(x_t)
    else:
        print('------HOT MODEL------')
        logit_model1 = LogisticRegression(C=0.1, verbose=0, max_iter=1000)
        logit_model1.fit(x, y)
        y_pred = logit_model1.predict(x_t)

    print(accuracy_score(y_t, y_pred))
    print(confusion_matrix(y_t, y_pred))
    print(classification_report(y_t, y_pred))

    if met == 'ndvi':
        with open('/content/sample_data/logit_model_ndvi.pkl', 'wb') as file:
            pickle.dump(logit_model, file)
    else:
        with open('/content/sample_data/logit_model_hot.pkl', 'wb') as file:
            pickle.dump(logit_model1, file)


------NDVI MODEL------
0.8716338
[[10032093   854489]
 [ 1071004  3042414]]
              precision    recall  f1-score   support

         0.0       0.90      0.92      0.91  10886582
         1.0       0.78      0.74      0.76   4113418

    accuracy                           0.87  15000000
   macro avg       0.84      0.83      0.84  15000000
weighted avg       0.87      0.87      0.87  15000000

------HOT MODEL------
0.9957556666666667
[[10808759    33756]
 [   29909  4127576]]
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00  10842515
         1.0       0.99      0.99      0.99   4157485

    accuracy                           1.00  15000000
   macro avg       0.99      0.99      0.99  15000000
weighted avg       1.00      1.00      1.00  15000000

